In [2]:
from mxnet import autograd, nd

### 梯度
![grad](grad.png)

In [3]:
x = nd.arange(4).reshape((4, 1))

In [4]:
x


[[0.]
 [1.]
 [2.]
 [3.]]
<NDArray 4x1 @cpu(0)>

In [6]:
# 为了求有关变量x的梯度,我们需要先调用attach_grad函数来申请存储梯度所需要的内存
x.attach_grad()

为了减少计算和内存开销,默认条件下MXNet不会记录用于求梯度
的计算。我们需要调用record函数来要求MXNet记录与求梯度有关的计算。

In [7]:
with autograd.record():
    y = 2 * nd.dot(x.T, x)

接下来我们可以通过调用backward函数自动求梯度。需
要注意的是,如果y不是一个标量,MXNet将默认先对y中元素求和得到新的变量,再求该变量有
关x的梯度。

In [8]:
y.backward()

In [9]:
# 函数 y = 2x ⊤ x 关于x 的梯度应为4x。现在我们来验证一下求出来的梯度是正确的。
assert(x.grad - 4 * x).norm().asscalar() == 0

In [14]:
x.grad


[[ 0.]
 [ 4.]
 [ 8.]
 [12.]]
<NDArray 4x1 @cpu(0)>

In [11]:
y


[[28.]]
<NDArray 1x1 @cpu(0)>

##### 训练模式和预测模式
从上面可以看出,在调用record函数后,MXNet会记录并计算梯度.此外,默认情况下还会将运行模式从预测模式转化为训练模式.这可以通过is_training函数来查看

In [12]:
print(autograd.is_training())
with autograd.record():
    print(autograd.is_training())

False
True


##### 对Python控制流求梯度

In [15]:
def f(a):
    b = a * 2
    while b.norm().asscalar() < 1000:
        b = b * 2
    if b.sum().asscalar() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [24]:
a = nd.random.normal(shape=1)
a.attach_grad()
with autograd.record():
    c = f(a)
c.backward()
a


[1.1839255]
<NDArray 1 @cpu(0)>

In [27]:
a.grad == c / a


[1.]
<NDArray 1 @cpu(0)>